PDF 문서는 한 문단을 알 수 없음

- 줄 바꿈은 다른 줄
- 단어가 잘릴 경우 알 수 없음
- 워드는 한 문단으로 인식하므로 워드가 추천됨

워드는 그냥 document loader에 넘기면 읽을 수 없음

- rtf라는 포맷으로 되어있음
- docx로 변경 필요

1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못 할 수 있고
    - 문서가 길면(인풋이 길면) 답변 생성이 오래 걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 떄, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

Text Splitter 는 여러 종류가 있음

- 일반적으로 문서는 RecursiveCharacterTextSplitter 혹은 CharacterTextSplitter를 사용
- RecursiveCharacterTextSplitter는 리스트로 구분자를 지정할 수 있는 반면 CharacterTextSplitter는 단일 문자만 지정 가능
- 그래서 RecursiveCharacterTextSplitter 가 성능이 더 좋음

Text Splitter

- chunk_size : 쪼갤 크기
- chunk_overlap : 겹치는 크기
    - 오버랩을 줘서 우리가 원하는 문서를 더 잘 가져올 수 있도록 함

langchain 은 체인이므로 더 효과적으로 retrieval 할 수 있음

> Retrieval QA Chain: Retrieval 하고 Question Answer

langchainhub 은 많은 기능들이 있지만, 프롬프트를 많이 줌

In [16]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [12]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 200,
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [13]:
len(document_list)

183

In [15]:
document_list[1]

Document(metadata={'source': './tax.docx'}, page_content='1. 구성원 간 이익의 분배비율이 정하여져 있고 해당 구성원별로 이익의 분배비율이 확인되는 경우\n\n2. 구성원 간 이익의 분배비율이 정하여져 있지 아니하나 사실상 구성원별로 이익이 분배되는 것으로 확인되는 경우\n\n④ 제3항에도 불구하고 해당 단체의 전체 구성원 중 일부 구성원의 분배비율만 확인되거나 일부 구성원에게만 이익이 분배되는 것으로 확인되는 경우에는 다음 각 호의 구분에 따라 소득세 또는 법인세를 납부할 의무를 진다.<신설 2018. 12. 31.>\n\n1. 확인되는 부분: 해당 구성원별로 소득세 또는 법인세에 대한 납세의무 부담\n\n2. 확인되지 아니하는 부분: 해당 단체를 1거주자 또는 1비거주자로 보아 소득세에 대한 납세의무 부담\n\n⑤ 제3항 및 제4항에도 불구하고 법인으로 보는 단체 외의 법인 아닌 단체에 해당하는 국외투자기구(투자권유를 하여 모은 금전 등을 가지고 재산적 가치가 있는 투자대상자산을 취득, 처분하거나 그 밖의 방법으로 운용하고 그 결과를 투자자에게 배분하여 귀속시키는 투자행위를 하는 기구로서 국외에서 설립된 기구를 말한다. 이하 같다)를 제119조의2제1항제2호에 따라 국내원천소득의 실질귀속자로 보는 경우 그 국외투자기구는 1비거주자로서 소득세를 납부할 의무를 진다.<신설 2018. 12. 31.>\n\n[전문개정 2009. 12. 31.]\n\n[제1조에서 이동, 종전 제2조는 제2조의2로 이동 <2009. 12. 31.>]\n\n\n\n제2조의2(납세의무의 범위) ① 제43조에 따라 공동사업에 관한 소득금액을 계산하는 경우에는 해당 공동사업자별로 납세의무를 진다. 다만, 제43조제3항에 따른 주된 공동사업자(이하 이 항에서 “주된 공동사업자”라 한다)에게 합산과세되는 경우 그 합산과세되는 소득금액에 대해서는 주된 공동사업자의 특수관계인은 같은 조 제2항에 따른 손익분배비율에 해당하는 그의 소득금액을 한도로 주된

In [17]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

load_dotenv()

embedding = UpstageEmbeddings(model='solar-embedding-1-large')

In [31]:
from langchain_chroma import Chroma

# croma 는 인메모리 벡터 디비
# persistent_directory: 파일로 저장
# collection_name: 테이블 이름
database = Chroma.from_documents(
    documents=document_list,
    embedding=embedding,
    collection_name= 'croma-tax',
    persist_directory='./chroma'
)

# database = Chroma.from_documents(collection_name= 'croma-tax', persist_directory='./chroma', embedding=embedding)

In [22]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=3) # k는 몇 개를 가져올지

In [24]:
retrieved_docs

[Document(id='7c8790ae-13c8-4d8a-9b05-0da0cbd8e10f', metadata={'source': './tax.docx'}, page_content='2. 2명인 경우: 연 55만원\n\n3. 3명 이상인 경우: 연 55만원과 2명을 초과하는 1명당 연 40만원을 합한 금액\n\n② 삭제<2017. 12. 19.>\n\n③ 해당 과세기간에 출산하거나 입양 신고한 공제대상자녀가 있는 경우 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제한다.<신설 2015. 5. 13., 2016. 12. 20.>\n\n1. 출산하거나 입양 신고한 공제대상자녀가 첫째인 경우: 연 30만원\n\n2. 출산하거나 입양 신고한 공제대상자녀가 둘째인 경우: 연 50만원\n\n3. 출산하거나 입양 신고한 공제대상자녀가 셋째 이상인 경우: 연 70만원\n\n④ 제1항 및 제3항에 따른 공제를 “자녀세액공제”라 한다.<신설 2015. 5. 13., 2017. 12. 19.>\n\n[본조신설 2014. 1. 1.]\n\n[종전 제59조의2는 제59조의5로 이동 <2014. 1. 1.>]\n\n\n\n제59조의3(연금계좌세액공제) ① 종합소득이 있는 거주자가 연금계좌에 납입한 금액 중 다음 각 호에 해당하는 금액을 제외한 금액(이하 “연금계좌 납입액”이라 한다)의 100분의 12[해당 과세기간에 종합소득과세표준을 계산할 때 합산하는 종합소득금액이 4천 500만원 이하(근로소득만 있는 경우에는 총급여액 5천 500만원 이하)인 거주자에 대해서는 100분의 15]에 해당하는 금액을 해당 과세기간의 종합소득산출세액에서 공제한다. 다만, 연금계좌 중 연금저축계좌에 납입한 금액이 연 600만원을 초과하는 경우에는 그 초과하는 금액은 없는 것으로 하고, 연금저축계좌에 납입한 금액 중 600만원 이내의 금액과 퇴직연금계좌에 납입한 금액을 합한 금액이 연 900만원을 초과하는 경우에는 그 초과하는 금액은 없는 것으로 한다. <개정 2014. 12. 23., 2015.

In [25]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()

In [26]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요.

[Context]
{retrieved_docs}

Question: {query}
"""

In [28]:
ai_message = llm.invoke(prompt)

In [32]:
print(ai_message.content)

연봉 5,000만원인 직장인의 소득세를 계산하기 위해서는 다음과 같은 단계를 거쳐야 합니다.

1. **근로소득공제**: 총급여액 5,000만원에서 공제액 2,500만원을 뺀 2,500만원이 근로소득이 됩니다.

2. **기본세율 적용**: 근로소득 2,500만원에 대해 기본세율을 적용합니다. 2023년 현재 소득세율은 다음과 같습니다.
   - 1,200만원 이하: 6%
   - 1,200만원 초과 4,600만원 이하: 15%

   따라서, 2,500만원에 대해 세금을 계산하면:
   - 1,200만원까지: 1,200만원 x 6% = 72만원
   - 1,200만원 초과분: 1,300만원 x 15% = 195만원

   총 근로소득세: 72만원 + 195만원 = 267만원

3. **근로소득세액공제**: 총급여액이 3,300만원 초과 7,000만원 이하이므로, 74만원 - [(총급여액 - 3,300만원) x 8/1,000] = 74만원 - 48만원 = 26만원

   따라서, 공제 후 근로소득세는 267만원 - 26만원 = 241만원입니다.

4. **기타 공제사항**: 질문에서 기타 공제사항(예: 표준세액공제, 자녀세액공제, 연금계좌세액공제 등)에 대한 정보가 없으므로, 본 답변에서는 이를 고려하지 않습니다.

따라서, 연봉 5,000만원인 직장인의 소득세는 기본세율 적용 후 근로소득세액공제를 고려하여 241만원이 됩니다.

**주의**: 실제 소득세 계산 시에는 다양한 공제사항이 추가로 적용될 수 있으므로, 정확한 계산을 위해서는 개인의 구체적인 상황을 고려해야 합니다.


In [33]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [34]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [36]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [38]:
ai_message = qa_chain.invoke({"query": query})

In [39]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 자녀세액공제는 55만원입니다. 이는 공제대상자녀가 2명인 경우에 해당하는 금액입니다.'}